In [1]:
%matplotlib inline
import sys
from collections import defaultdict, Counter
import networkx as nx
import numpy as np
import pylab as plt
import math
from itertools import chain
#import matplotlib.cm as cm
import statsmodels.api as sm
from os import listdir
from scipy.stats.stats import pearsonr
from sklearn.externals.six.moves.urllib.request import urlopen
import json
import random
import itertools
from copy import deepcopy
import time
import scipy as sp
from scipy.sparse import csr_matrix
from scipy.stats import entropy
import matplotlib.cm as cm
from numpy.random import choice
import itertools
from sklearn import cluster, covariance, manifold
import operator
import itertools
from datetime import datetime as dt
import matplotlib.mlab as mlab
from scipy.stats import norm
import re
from scipy import stats
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.mstats import zscore



 Import Hawkes python module and the content will be in a subfolder labeled hawkes:
https://github.com/stmorse/hawkes

In [2]:
import sys
sys.path.append('hawkes/')
from MHP import MHP

### Load Time series
Hawkes only looks at the events at the same time series

In [3]:
#1. Load DSDE jobs, and select skills that are in WoS
J={}
with open('time_series1.txt','r', encoding ='latin') as f:
    for line in f:
        skill=line.split('[')[0].strip().lower() # extract skill
        vs=line.split('[')[1].split(']')[0].split(',') # vector : extract years + frequency - all content split by commas
        dic=dict((int(i.split(':')[0]),int(i.split(':')[1])) for i in vs) # split year and freq
        # KEY is skill, VALUE is year, freq 
        J[skill]=dic

### Data Transformation 
Run this block to replace zero frequencies by 0.00001 

In [4]:
#JOBS
JW={}
for i in J:
    JW[i] = J[i]


In [5]:
### Add non-zero values
for key, value in JW.items():

    for i in JW[key]:
        if JW[key][i]==0:
            JW[key][i]=0.00001

In [6]:
#3.  split J by year and find total per year
## JOB DATA TRANSFORMATION USE JW
JT=defaultdict(int) # initializing first dictionary for collection/list
for i in JW:
    for year in JW[i]:
        JT[year]+=JW[i][year]

JN={}
for i in JW:
    dic=dict((k,v/float(JT[k])) for k,v in JW[i].items() if 2010<=k<=2016 and v!=0)#if 2010<=k<=2016 and v!=0 ) 

    if len(dic)==7:
        JN[i]=dic

In [19]:
# total frequencies per skills for influence graph
JS=defaultdict(int)
for i in J:
    for year in J[i]:
       # JS[i]+=float(str(round(J[i][year],0)))
        JS[i]+=int(J[i][year])

### TOP Frequency Skills
We select to 50 skills (words) by frequency that will be used in Hawkes process

In [9]:
#select top DSDE skills (top 50 still Bursing skills )
N=50

topSkills=sorted(list(zip(*sorted([(k,v) for k,v in J.items()],key=lambda x:-sum(x[1].values()))))[0][:N])


In [10]:
DSDE=dict((k,v) for k,v in JN.items() if k in topSkills)
DSDE1={}
for s in DSDE:
    dic=DSDE[s]
    t=sum(dic.values())
    DSDE1[s]=dict((k,v/t) for k,v in dic.items())

## Hawkes Process

Read - Steven Morse blog and thesis - https://stmorse.github.io/journal/Hawkes-python.html 

From personal communication with Steven:

"The MHP.generate_seq() method returns an array with shape (N, 2), each row an event, col 0 is time, col 1 is the label of the sequence where it occurred.  If you have already run generate_seq(), you should be able to then call EM() and get estimated parameters, EM() uses the same format and the data is saved as a variable of that class instance."

In [11]:
# simulate data for Hawkes Process estimation 
n=10# expanding 
Seq=[]
skills=DSDE1.keys()
nSkills=len(skills)
for year in range(2010,2017):
   # print(year)
    for i in range(n):
       # print(i)
        ps=[DSDE1[s][year] for s in skills]
       # print(len(ps))
       # print(ps)
        rp=np.random.uniform(0,1,len(ps))
       # print(rp)
        seq=(rp<ps).astype(int)
       # print(seq)
        Seq.append(seq)
Seq=np.array(Seq)
for i in range(Seq.shape[1]):
    v=Seq[:,i]
    v[v==1]=i+1
td=[]
t=0
#print(len(Seq))

for i in Seq:
   #print(i)
    for j in i:
        t+=random.random()
        if j!=0:
            td.append([t,j])
          # print(t,j)
       # else:
           # print(j)
td=np.array(td)
td[:,1]-=1

In [228]:
#TEST

# simulate data for Hawke Process estimation 
n=10# expanding 
Seq=[]
skills=DSDETest.keys()
nSkills=len(skills)
for year in range(2010,2017):
   # print(year)
    for i in range(n):
       # print(i)
        ps=[DSDETest[s][year] for s in skills]
       # print(len(ps))
       # print(ps)
        rp=np.random.uniform(0,1,len(ps))
       # print(rp)
        seq=(rp<ps).astype(int)
       # print(seq)
        Seq.append(seq)
Seq=np.array(Seq)
for i in range(Seq.shape[1]):
    v=Seq[:,i]
    v[v==1]=i+1
td=[]
t=0
#print(len(Seq))

for i in Seq:
   #print(i)
    for j in i:
        t+=random.random()
        if j!=0:
            td.append([t,j])
          # print(t,j)
       # else:
           # print(j)
td=np.array(td)
td[:,1]-=1

In [12]:
# initialization models 

mhat = np.random.uniform(0,1, size=nSkills)
ahat = np.random.uniform(0,1, size=(nSkills,nSkills))
w = 3
P = MHP(mu=mhat, alpha=ahat, omega=w)
#P.generate_seq(60)
P.data=td
#P.plot_events()
len(P.data)

Max eigenvalue: 24.66915
(WARNING) Unstable.


455

In [13]:
start=time.time()
M=P.EM(ahat, mhat, w)[0]
duration=time.time()-start
duration

After ITER 0 (old: -10000.000 new: -1.110)
 terms -454.9621, 404.8030, 50.1970
After ITER 10 (old: -1.110 new: -4.894)
 terms -2100.3062, 328.6592, 126.3408
Reached stopping criterion. (Old: -4.894 New: -4.895)


2.773644208908081

In [14]:
# Graph 
mM=np.mean(M)
G=nx.from_numpy_matrix(M>mM) # Return a graph from numpy matrix.
mapping=dict(zip(range(nSkills),skills))
H=nx.relabel_nodes(G,mapping)
pos=nx.spring_layout(H)